In [1]:
import os
from zenml.repo import Repository
from zenml.datasources import CSVDatasource
from zenml.pipelines import TrainingPipeline
from zenml.steps.evaluator import TFMAEvaluator
from zenml.steps.preprocesser import StandardPreprocesser
from zenml.steps.split import RandomSplit
from zenml.steps.trainer import TFFeedForwardTrainer
from zenml.repo import Repository, ArtifactStore
from zenml.utils.naming_utils import transformed_label_name
from zenml.steps.deployer import GCAIPDeployer
from zenml.steps.deployer import CortexDeployer
from examples.cortex.predictor.tf import TensorFlowPredictor
from zenml.backends.orchestrator import OrchestratorGCPBackend
from zenml.metadata import MySQLMetadataStore
from zenml.backends.processing import ProcessingDataFlowBackend
from zenml.backends.training import SingleGPUTrainingGCAIPBackend

# Set up some variables

In [2]:
GCP_BUCKET=os.getenv('GCP_BUCKET')
GCP_PROJECT=os.getenv('GCP_PROJECT')
GCP_REGION=os.getenv('GCP_REGION')
GCP_CLOUD_SQL_INSTANCE_NAME=os.getenv('GCP_CLOUD_SQL_INSTANCE_NAME')
MODEL_NAME=os.getenv('MODEL_NAME')
CORTEX_ENV=os.getenv('CORTEX_ENV')
MYSQL_DB=os.getenv('MYSQL_DB')
MYSQL_USER=os.getenv('MYSQL_USER')
MYSQL_PWD=os.getenv('MYSQL_PWD')
MYSQL_PORT=os.getenv('MYSQL_PORT')
MYSQL_HOST=os.getenv('MYSQL_HOST')
CONNECTION_NAME = f'{GCP_PROJECT}:{GCP_REGION}:{GCP_CLOUD_SQL_INSTANCE_NAME}'
TRAINING_JOB_DIR = os.path.join(GCP_BUCKET, 'gcp_gcaip_training/staging')

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [4]:
repo: Repository = Repository.get_instance()
    
artifact_store = ArtifactStore(os.path.join(GCP_BUCKET, 'all_feature_demo'))

# Create first pipeline

In [6]:
training_pipeline = TrainingPipeline(name='Experiment 1')

2021-04-27 10:46:30,928 — zenml.pipelines.base_pipeline — INFO — Pipeline Experiment 1 created.


#### Add a datasource. This will automatically track and version it.

In [7]:
try:
    ds = CSVDatasource(name='Pima Indians Diabetes', path='gs://zenml_quickstart/diabetes.csv')
except:
    repo: Repository = Repository.get_instance()
    ds = repo.get_datasource_by_name('Pima Indians Diabetes')
training_pipeline.add_datasource(ds)

2021-04-27 10:46:32,371 — zenml.datasources.base_datasource — INFO — Datasource Pima Indians Diabetes created.


#### Add a split step to partition data into train and eval

In [8]:
training_pipeline.add_split(RandomSplit(split_map={'train': 0.7, 'eval': 0.2, 'test':0.1}))

#### Add a preprocessing step to transform data to be ML-capable

In [9]:
training_pipeline.add_preprocesser(
    StandardPreprocesser(
        features=['times_pregnant', 'pgc', 'dbp', 'tst', 'insulin', 'bmi',
                  'pedigree', 'age'],
        labels=['has_diabetes'],
        overwrite={'has_diabetes': {
            'transform': [{'method': 'no_transform', 'parameters': {}}]}}
    ))

#### Add a trainer which defines model and training

In [10]:
training_pipeline.add_trainer(TFFeedForwardTrainer(
    loss='binary_crossentropy',
    last_activation='sigmoid',
    output_units=1,
    metrics=['accuracy'],
    epochs=5))

#### Add an evaluator to calculate slicing metrics

In [11]:
training_pipeline.add_evaluator(
    TFMAEvaluator(slices=[['has_diabetes']],
                  metrics={transformed_label_name('has_diabetes'):
                     ['binary_crossentropy', 'binary_accuracy']}))

#### Run and evaluate

In [12]:
training_pipeline.run()

2021-04-27 10:46:38,771 — zenml.pipelines.training_pipeline — INFO — Datasource Pima Indians Diabetes has no commits. Creating the first one..
2021-04-27 10:46:38,774 — zenml.pipelines.base_pipeline — INFO — Pipeline 1619513198774 created.
2021-04-27 10:46:38,821 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataGen is running.
2021-04-27 10:46:39,401 — zenml.datasources.csv_datasource — INFO — Matched 1: ['gs://zenml_quickstart/diabetes.csv']
2021-04-27 10:46:39,406 — zenml.datasources.csv_datasource — INFO — Using header from file: gs://zenml_quickstart/diabetes.csv.
2021-04-27 10:46:39,797 — zenml.datasources.csv_datasource — INFO — Header: ['times_pregnant', 'pgc', 'dbp', 'tst', 'insulin', 'bmi', 'pedigree', 'age', 'has_diabetes'].


2021-04-27 10:46:42,833 — apache_beam.internal.gcp.auth — WARNING — Unable to find default credentials to use: The Application Default Credentials are not available. They are available if running in Google Compute Engine. Otherwise, the environment variable GOOGLE_APPLICATION_CREDENTIALS must be defined pointing to a file defining the credentials. See https://developers.google.com/accounts/docs/application-default-credentials for more information.
Connecting anonymously.
2021-04-27 10:46:43,277 — apache_beam.runners.interactive.interactive_environment — WARNING — Dependencies required for Interactive Beam PCollection visualization are not available, please use: `pip install apache-beam[interactive]` to install necessary dependencies to enable all data visualization features.


2021-04-27 10:46:45,237 — apache_beam.io.tfrecordio — WARNING — Couldn't find python-snappy so the implementation of _TFRecordUtil._masked_crc32c is not as fast as it could be.


2021-04-27 10:46:45,473 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataGen is finished.
2021-04-27 10:46:45,481 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataStatistics is running.
2021-04-27 10:46:46,799 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataStatistics is finished.
2021-04-27 10:46:46,802 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component DataSchema is running.


2021-04-27 10:46:46,843 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow_data_validation/utils/stats_util.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


...

      1/Unknown - 0s 137us/step - loss: 0.5849 - accuracy: 0.7500

2021-04-27 10:47:06,569 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
2021-04-27 10:47:06,583 — tensorflow — WARNING — Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0018s vs `on_train_batch_end` time: 0.0145s). Check your callbacks.


64/64 [==============================]0.6183 - accuracy: 0.6481  - 0s 2ms/step - loss: 0.6151 - accuracy: 0.676 - 0s 5ms/step - loss: 0.6124 - accuracy: 0.6797 - val_loss: 0.6636 - val_accuracy: 0.6726
Epoch 2/5
64/64 [==============================] - ETA: 0s - loss: 0.4887 - accuracy: 0.62 - ETA: 0s - loss: 0.5813 - accuracy: 0.71 - 0s 2ms/step - loss: 0.5849 - accuracy: 0.7090 - val_loss: 0.6420 - val_accuracy: 0.7083
Epoch 3/5
64/64 [==============================] - ETA: 0s - loss: 0.4069 - accuracy: 1.00 - ETA: 0s - loss: 0.5569 - accuracy: 0.73 - 0s 3ms/step - loss: 0.5682 - accuracy: 0.7207 - val_loss: 0.6189 - val_accuracy: 0.7202
Epoch 4/5
64/64 [==============================] - ETA: 0s - loss: 0.5433 - accuracy: 0.62 - ETA: 0s - loss: 0.5259 - accuracy: 0.76 - 0s 3ms/step - loss: 0.5430 - accuracy: 0.7461 - val_loss: 0.5854 - val_accuracy: 0.7202
Epoch 5/5
64/64 [==============================] - ETA: 0s - loss: 0.6116 - accuracy: 0.75 - ETA: 0s - loss: 0.5374 - accuracy: 0

/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['insulin', 'pedigree', 'has_diabetes', 'times_pregnant', 'pgc', 'age', 'dbp', 'has_diabetes_xl', 'bmi', 'tst'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])
/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['age', 'bmi', 'dbp', 'has_diabetes', 'has_diabetes_xl', 'insulin', 'pedigree', 'pgc', 'times_pregnant', 'tst'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])
2021-04-27 10:47:11,246 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/training/tracking/tracking.py:111: Model.state_updates (from tensorflow.

2021-04-27 10:47:11,798 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Trainer is finished.
2021-04-27 10:47:11,802 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Evaluator is running.
2021-04-27 10:47:16,256 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Evaluator is finished.


In [13]:
training_pipeline.view_statistics(magic=True)

2021-04-27 10:48:50,627 — zenml.pipelines.training_pipeline — INFO — Viewing statistics. If magic=False then a new window will open up with a notebook for evaluation. If magic=True, then an attempt will be made to append to the current notebook.


In [ ]:
training_pipeline.evaluate(magic=True)

#### Inspect datasource

In [14]:
datasources = repo.get_datasources()
datasource = datasources[0]
print(datasource)

args:
  path: gs://zenml_quickstart/diabetes.csv
  schema: null
id: 9a9e2f4e-8b97-4fe0-bff4-3fc92b029ebb
name: Pima Indians Diabetes
source: zenml.datasources.csv_datasource.CSVDatasource@zenml_0.3.7



In [15]:
df = datasource.sample_data()
df.head()

2021-04-27 10:48:57,620 — zenml.utils.post_training.post_training_utils — INFO — Converting dataset to Pandas DataFrame..


,age,bmi,dbp,has_diabetes,insulin,pedigree,pgc,times_pregnant,tst
0,50,33.599998,72,1,0,0.627,148,6,35
1,31,26.600000,66,0,0,0.351,85,1,29
2,32,23.299999,64,1,0,0.672,183,8,0
3,21,28.100000,66,0,94,0.167,89,1,23
4,33,43.099998,40,1,168,2.288,137,0,35


In [16]:
df.shape

(768, 9)

In [17]:
df.columns

Index(['age', 'bmi', 'dbp', 'has_diabetes', 'insulin', 'pedigree', 'pgc',
       'times_pregnant', 'tst'],
      dtype='object')

## Skip preprocessing with your next (warm-starting) pipeline

#### Clone first experiment and only change one hyper-parameter

In [18]:
training_pipeline_2 = training_pipeline.copy('Experiment 2')
training_pipeline_2.add_trainer(TFFeedForwardTrainer(
    loss='binary_crossentropy',
    last_activation='sigmoid',
    output_units=1,
    metrics=['accuracy'],
    epochs=20))

2021-04-27 10:48:59,607 — zenml.pipelines.base_pipeline — INFO — Pipeline Experiment 2 created.


In [19]:
training_pipeline_2.run()

2021-04-27 10:49:00,340 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataGen is running.
2021-04-27 10:49:00,375 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataGen is finished.
2021-04-27 10:49:00,378 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataSchema is running.
2021-04-27 10:49:00,408 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataSchema is finished.
2021-04-27 10:49:00,410 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataStatistics is running.
2021-04-27 10:49:00,435 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataStatistics is finished.
2021-04-27 10:49:00,438 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component SplitGen is running.
2021-04-27 10:49:00,473 — zenml.backe

/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['age', 'bmi', 'dbp', 'has_diabetes', 'insulin', 'pedigree', 'pgc', 'times_pregnant', 'tst'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


      1/Unknown - 0s 81us/step - loss: 0.6498 - accuracy: 0.6250

2021-04-27 10:49:01,153 — tensorflow — WARNING — Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0012s vs `on_train_batch_end` time: 0.0323s). Check your callbacks.


64/64 [==============================]0.7148 - accuracy: 0.5179 - 0s 2ms/step - loss: 0.7067 - accuracy: 0.569 - 0s 6ms/step - loss: 0.6941 - accuracy: 0.5957 - val_loss: 0.6455 - val_accuracy: 0.6369
Epoch 2/20
64/64 [==============================] - ETA: 0s - loss: 0.5189 - accuracy: 0.62 - ETA: 0s - loss: 0.6354 - accuracy: 0.62 - ETA: 0s - loss: 0.6263 - accuracy: 0.62 - 0s 3ms/step - loss: 0.6238 - accuracy: 0.6348 - val_loss: 0.5990 - val_accuracy: 0.6786
Epoch 3/20
64/64 [==============================] - ETA: 0s - loss: 0.5659 - accuracy: 0.75 - ETA: 0s - loss: 0.5910 - accuracy: 0.69 - 0s 2ms/step - loss: 0.5820 - accuracy: 0.6934 - val_loss: 0.5732 - val_accuracy: 0.7024
Epoch 4/20
64/64 [==============================] - ETA: 0s - loss: 0.6676 - accuracy: 0.37 - ETA: 0s - loss: 0.5665 - accuracy: 0.69 - 0s 2ms/step - loss: 0.5618 - accuracy: 0.6914 - val_loss: 0.5539 - val_accuracy: 0.7143
Epoch 5/20
64/64 [==============================] - ETA: 0s - loss: 0.7364 - accuracy

/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['tst', 'times_pregnant', 'pgc', 'age', 'bmi', 'dbp', 'pedigree', 'insulin', 'has_diabetes_xl', 'has_diabetes'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])
/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow/python/keras/engine/functional.py:543: UserWarning: Input dict contained keys ['age', 'bmi', 'dbp', 'has_diabetes', 'has_diabetes_xl', 'insulin', 'pedigree', 'pgc', 'times_pregnant', 'tst'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


2021-04-27 10:49:07,145 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Trainer is finished.
2021-04-27 10:49:07,149 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Evaluator is running.


2021-04-27 10:49:09,322 — tensorflow — WARNING — 5 out of the last 5 calls to <function recreate_function.<locals>.restored_function_body at 0x7f63fc57b620> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has experimental_relax_shapes=True option that relaxes argument shapes that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/tutorials/customization/performance#python_or_tensor_args and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
2021-04-27 10:49:10,021 — tensorflow — WARNING — 6 out of the last 6 calls to <function recreate_function.<locals>.restored_function_body at 0x7f6414547598> triggered tf.function retracing. Tracing is expensive and th

2021-04-27 10:49:11,245 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Evaluator is finished.


In [20]:
training_pipeline_2.evaluate(magic=True)

2021-04-27 10:49:11,279 — zenml.pipelines.training_pipeline — INFO — Evaluating pipeline. If magic=False then a new window will open up with a notebook for evaluation. If magic=True, then an attempt will be made to append to the current notebook.


## Post-training

#### Verify theres still only one datasource

In [ ]:
datasources = repo.get_datasources()
print(f"We have {len(datasources)} datasources")

#### Compare pipelines

In [ ]:
repo.compare_training_runs()

# Easily train on the cloud

In [7]:
training_pipeline_3 = training_pipeline.copy('Experiment 3')

# Add a trainer with a GCAIP backend
training_backend = SingleGPUTrainingGCAIPBackend(
    project=GCP_PROJECT,
    job_dir=TRAINING_JOB_DIR
)

training_pipeline.add_trainer(TFFeedForwardTrainer(
    loss='binary_crossentropy',
    last_activation='sigmoid',
    output_units=1,
    metrics=['accuracy'],
    epochs=20).with_backend(training_backend))

training_pipeline_3.run(artifact_store=artifact_store)

2021-04-27 10:58:22,184 — zenml.pipelines.base_pipeline — INFO — Pipeline Experiment 3 created.
2021-04-27 10:58:22,194 — zenml.pipelines.base_pipeline — WARNING — Changing the metadata_store or artifact_store might cause your pipelines to be non-reproducible and non-comparable.
2021-04-27 10:58:22,384 — zenml.backends.training.training_gcaip_backend — INFO — Launching GCAIP Trainer job train_1619513827
2021-04-27 10:58:22,386 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataGen is running.
2021-04-27 10:58:22,440 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataGen is finished.
2021-04-27 10:58:22,443 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataSchema is running.
2021-04-27 10:58:22,474 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataSchema is finished.
2021-04-27 10:58:22,477 — zenml.backends.

2021-04-27 10:58:26,304 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow_data_validation/utils/stats_util.py:247: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
2021-04-27 10:58:26,346 — apache_beam.runners.interactive.interactive_environment — WARNING — Dependencies required for Interactive Beam PCollection visualization are not available, please use: `pip install apache-beam[interactive]` to install necessary dependencies to enable all data visualization features.


2021-04-27 10:58:26,999 — apache_beam.io.tfrecordio — WARNING — Couldn't find python-snappy so the implementation of _TFRecordUtil._masked_crc32c is not as fast as it could be.


2021-04-27 10:58:33,267 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component SplitGen is finished.
2021-04-27 10:58:33,274 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component SplitStatistics is running.
2021-04-27 10:58:48,231 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component SplitStatistics is finished.
2021-04-27 10:58:48,233 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component SplitSchema is running.
2021-04-27 10:58:54,847 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component SplitSchema is finished.
2021-04-27 10:58:54,854 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Transform is running.


2021-04-27 10:59:03,513 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tfx/components/transform/executor.py:541: Schema (from tensorflow_transform.tf_metadata.dataset_schema) is deprecated and will be removed in a future version.
Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
2021-04-27 10:59:03,756 — tensorflow — WARNING — From /home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/tensorflow_transform/tf_utils.py:261: Tensor.experimental_ref (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use ref() instead.
2021-04-27 10:59:04,207 — tensorflow — WARNING — TFT beam APIs accept both the TFXIO format and the instance dict format now. There is no need to set use_tfxio any more and it will be removed soon.
2021-04-27 10:59:04,231 — root — WARNING — This output type hint will be ignored and not us

2021-04-27 11:00:46,466 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Transform is finished.
2021-04-27 11:00:46,473 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component Trainer is running.


2021-04-27 11:00:54,279 — googleapiclient.discovery_cache — WARNING — file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapi

RuntimeError: Job 'projects/core-engine/jobs/ce_train_1619513827' did not succeed.  Detailed response {'jobId': 'ce_train_1619513827', 'trainingInput': {'scaleTier': 'CUSTOM', 'masterType': 'n1-standard-4', 'region': 'europe-west1', 'jobDir': 'gs://zenmlartifactstore/gcp_gcaip_training/staging', 'pythonVersion': '3.7', 'masterConfig': {'acceleratorConfig': {'count': '1', 'type': 'NVIDIA_TESLA_K80'}, 'imageUri': 'eu.gcr.io/maiot-zenml/zenml:cuda-0.3.7', 'containerCommand': ['python', '-m', 'tfx.scripts.run_executor', '--executor_class_path', 'tfx.components.trainer.executor.GenericExecutor', '--inputs', '{"examples": [{"artifact": {"id": "22", "type_id": "5", "uri": "gs://zenmlartifactstore/all_feature_demo/c23c99da5684ed052ebd33a635d37715/Transform/transformed_examples/36", "properties": {"split_names": {"string_value": "[\\"eval\\", \\"test\\", \\"train\\"]"}}, "custom_properties": {"name": {"string_value": "transformed_examples"}, "state": {"string_value": "published"}, "producer_com

# Orchestrate every step on the Cloud

In [8]:
training_pipeline_4 = training_pipeline.copy('Experiment 4')

# Define the metadata store
metadata_store = MySQLMetadataStore(
    host=MYSQL_HOST,
    port=int(MYSQL_PORT),
    database=MYSQL_DB,
    username=MYSQL_USER,
    password=MYSQL_PWD,
)


# Define the orchestrator backend
orchestrator_backend = OrchestratorGCPBackend(
    cloudsql_connection_name=CONNECTION_NAME,
    project=GCP_PROJECT,
    preemptible=True,  # reduce costs by using preemptible instances
    machine_type='n1-standard-4',
    gpu='nvidia-tesla-k80',
    gpu_count=1,
)


# Run the pipeline
training_pipeline_4.run(
    backend=orchestrator_backend,
    metadata_store=metadata_store,
    artifact_store=artifact_store,
)

2021-04-27 12:22:19,311 — zenml.pipelines.base_pipeline — INFO — Pipeline Experiment 4 created.


2021-04-27 12:22:19,320 — googleapiclient.discovery_cache — WARNING — file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapi

2021-04-27 12:22:20,880 — zenml.pipelines.base_pipeline — WARNING — Changing the metadata_store or artifact_store might cause your pipelines to be non-reproducible and non-comparable.
2021-04-27 12:22:20,886 — zenml.pipelines.base_pipeline — WARNING — Changing the metadata_store or artifact_store might cause your pipelines to be non-reproducible and non-comparable.
2021-04-27 12:22:20,914 — zenml.backends.orchestrator.gcp.orchestrator_gcp_backend — INFO — Orchestrating pipeline on GCP..
2021-04-27 12:22:23,761 — zenml.backends.orchestrator.gcp.orchestrator_gcp_backend — INFO — Created tar of current repository at: /home/hamza/workspace/maiot/github_temp/zenml/.zenml/zenml_working_1619518940.tar.gz
2021-04-27 12:22:55,459 — zenml.backends.orchestrator.gcp.orchestrator_gcp_backend — INFO — Copied tar to artifact store at: gs://zenmlartifactstore/all_feature_demo/staging/zenml_working_1619518940.tar.gz
2021-04-27 12:22:55,468 — zenml.backends.orchestrator.gcp.orchestrator_gcp_backend — IN

2021-04-27 12:22:55,481 — googleapiclient.discovery_cache — WARNING — file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapi

2021-04-27 12:22:58,294 — zenml.backends.orchestrator.gcp.orchestrator_gcp_backend — INFO — Launched instance zenml-1619518939 with ID: 345204829920581871
2021-04-27 12:22:58,297 — zenml.backends.orchestrator.gcp.orchestrator_gcp_backend — INFO — View logs at: https://console.cloud.google.com/logs/query;query=logName%3D%22projects%2Fcore-engine%2Flogs%2Fgcplogs-docker-driver%22%0Aresource.labels.instance_id%3D%22345204829920581871%22?project=core-engine&folder=true&query=%0A


# Add a deployer step with different integrations

## Option 1: Deploy to Google Cloud AI Platform

In [12]:
training_pipeline_5 = training_pipeline.copy('Experiment 5')
training_pipeline_5.add_deployment(
    GCAIPDeployer(
        project_id=GCP_PROJECT,
        model_name=MODEL_NAME,
    )
)

training_pipeline_5.run(artifact_store=artifact_store)

2021-04-27 12:39:52,535 — zenml.pipelines.base_pipeline — INFO — Pipeline Experiment 11 created.
2021-04-27 12:39:52,537 — zenml.steps.deployer.gcaip_deployer — WARNING — When using GCAIPDeployer, please ensure that the Artifact Store is a Google Cloud Bucket.
2021-04-27 12:39:52,538 — zenml.pipelines.base_pipeline — WARNING — Changing the metadata_store or artifact_store might cause your pipelines to be non-reproducible and non-comparable.
2021-04-27 12:39:52,550 — zenml.steps.deployer.gcaip_deployer — WARNING — When using GCAIPDeployer, please ensure that the Artifact Store is a Google Cloud Bucket.
2021-04-27 12:39:52,799 — zenml.backends.training.training_gcaip_backend — INFO — Launching GCAIP Trainer job train_1619513827
2021-04-27 12:39:52,801 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataGen is running.
2021-04-27 12:39:52,848 — zenml.backends.orchestrator.base.zenml_local_orchestrator — INFO — Component ImporterNode.DataGen is f

2021-04-27 12:40:00,866 — googleapiclient.discovery_cache — WARNING — file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hamza/.virtualenvs/zenml_main/lib/python3.6/site-packages/googleapi

HttpError: <HttpError 409 when requesting https://ml.googleapis.com/v1/projects/core-engine/jobs?alt=json returned "Field: job.job_id Error: A job with this id already exists.". Details: "[{'@type': 'type.googleapis.com/google.rpc.BadRequest', 'fieldViolations': [{'field': 'job.job_id', 'description': 'A job with this id already exists.'}]}]">

## Option 2: Deploy to Kubernetes via Cortex

In [ ]:
training_pipeline = repo.get_pipeline_by_name('Experiment 1')

In [ ]:
training_pipeline_6 = training_pipeline.copy('Experiment 7')

# Add cortex deployer
api_config = {
    "name": MODEL_NAME,
    "kind": "RealtimeAPI",
    "predictor": {
        "type": "tensorflow",
        "models": {"signature_key": "serving_default"}}
}
training_pipeline.add_deployment(
    CortexDeployer(
        env=CORTEX_ENV,
        api_config=api_config,
        predictor=TensorFlowPredictor,
    )
)

training_pipeline_6.run(artifact_store=artifact_store)